**1.Load the requirement for creating the model.**

In [1]:
#!pip install tensorflow
#!pip install keras
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
import sys
import numpy as np, math
import pandas as pd

Using TensorFlow backend.


**2. Insert the data into code as Pandas dataframe.**

In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# The credentials will automatically be inserted when you insert the data into the code.
client_df9b53d4353f4ac684f6ea7d76d4cf1d = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='',
    ibm_auth_endpoint="",
    config=Config(signature_version='oauth'),
    endpoint_url='')

body = client_df9b53d4353f4ac684f6ea7d76d4cf1d.get_object(Bucket='fallclassification-donotdelete-pr-9niekv744xksho',Key='Label_test_lab.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,AvgX,AvgY,AvgZ,MedianX,MedianY,MedianZ,StdX,StdY,StdZ,SkewX,...,MeanTA,StdTA,SkewTA,KurtosisTA,MeanMag,StdMag,MinMag,MaxMag,DiffMinMaxMag,Label
0,0.428841,0.948442,0.410905,0.430634,0.952755,0.413757,0.005826,0.004746,0.005716,0.013313,...,0.925010,0.005027,0.567399,0.051587,0.273077,0.005233,0.985105,0.024857,0.005990,0
1,0.524837,0.934668,0.457115,0.513224,0.952505,0.446025,0.130639,0.078951,0.086613,0.043425,...,0.940625,0.065808,0.355957,0.109859,0.212789,0.090918,0.757186,0.042609,0.087951,1
2,0.844126,0.509207,0.673963,0.830995,0.487599,0.608691,0.276552,0.826460,0.254368,0.052013,...,0.536324,0.779088,0.487466,0.006794,0.443777,0.481026,0.529465,0.612727,0.573070,1
3,0.878770,0.275381,0.695474,0.892626,0.332303,0.724622,0.144757,0.246744,0.095654,0.056502,...,0.346887,0.287497,0.424815,0.020677,0.247056,0.070410,0.734768,0.074681,0.118150,1
4,0.931578,0.426702,0.740917,0.924685,0.429348,0.754027,0.051384,0.048115,0.023049,0.184864,...,0.460294,0.049495,0.420042,0.110924,0.255132,0.048531,0.938186,0.089877,0.067658,0


**3. Create, train, and evaluate the Keras Multi-layer Perceptron (MLP) model.**

In [4]:
dataset = df_data_1.values
# fix random seed for reproducibility
np.random.seed(7)

# split into input (X) and output (Y) variables
xvalues = dataset[:,0:-1]
yvalues = dataset[:,-1]
#print(xvalues[0:5,:])

# create model
numCol = xvalues.shape[1]
H1 = math.ceil(numCol/2)
#H2 = math.ceil(H1/2)
model = Sequential()
model.add(Dense(numCol, input_dim = numCol, activation='relu'))
model.add(Dense(H1, activation='relu'))
#model.add(Dense(H2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(xvalues, yvalues, epochs=20, batch_size=10)

# evaluate the model
scores = model.evaluate(xvalues, yvalues)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/20
7670/7670 [==============================] - 15s 2ms/step - loss: 0.1589 - acc: 0.9386
Epoch 2/20
7670/7670 [==============================] - 11s 1ms/step - loss: 0.0970 - acc: 0.9623
Epoch 3/20
7670/7670 [==============================] - 14s 2ms/step - loss: 0.0914 - acc: 0.9635
Epoch 4/20
7670/7670 [==============================] - 13s 2ms/step - loss: 0.0879 - acc: 0.9641
Epoch 5/20
7670/7670 [==============================] - 14s 2ms/step - loss: 0.0878 - acc: 0.9635
Epoch 6/20
7670/7670 [==============================] - 14s 2ms/step - loss: 0.0841 - acc: 0.9632
Epoch 7/20
7670/7670 [==============================] - 14s 2ms/step - loss: 0.0825 - acc: 0.9648
Epoch 8/20
7670/7670 [==============================] - 14s 2ms/step - loss: 0.0815 - acc: 0.9651
Epoch 9/20
7670/7670 [==============================] - 14s 2ms/step - loss: 0.0796 - acc: 0.9662
Epoch 10/20
7670/7670 [==============================] - 13s 2ms/step - loss: 0.0795 - acc: 0.9658
Epoch 11/20
7670/76

**4. Save the trained MLP model in .h5 format.**

In [5]:
from contextlib import suppress
import os

filename = 'model_oct19.h5'

# Delete a duplicate file if exists.
with suppress(OSError):
    os.remove(filename)
    
model.save(filename)
print("Saved model in h5")

Saved model in h5


**5. Compress the .h5 model as .tgz**

In [6]:
#compress keras model
tar_filename = filename + '.tgz'
cmdstring = 'tar -zcvf ' + tar_filename + ' ' + filename
os.system(cmdstring)

0

**Showing the content of the local file system.**

In [7]:
!ls

model_oct19.h5	model_oct19.h5.tgz


**6. Store the model in IBM Watson machine learning repository**

In [11]:
# use your Watson machine learning credentials here
from watson_machine_learning_client import WatsonMachineLearningAPIClient
client = WatsonMachineLearningAPIClient( {
  "apikey": "",
  "iam_apikey_description": "",
  "iam_apikey_name": "",
  "iam_role_crn": "",
  "iam_serviceid_crn": "",
  "instance_id": "",
  "url": ""
} )
metadata = {
    client.repository.ModelMetaNames.NAME: "keras model oct19_1",
    client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow",
    client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.13",
    client.repository.ModelMetaNames.RUNTIME_NAME: 'python',
    client.repository.ModelMetaNames.RUNTIME_VERSION: '3.6',
    client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES: [{'name':'keras', 'version': '2.2.4'}]
}
model_details = client.repository.store_model( tar_filename, meta_props=metadata, training_data = None  )
#training_data=xvalues, training_target=yvalues


**7. Deploy the stored model as a web service.**

In [12]:
model_id = client.repository.get_model_uid(model_details)
model_deployment_details = client.deployments.create(model_id, name="keras model oct19_1" )



#######################################################################################

Synchronous deployment creation for uid: 'e5e26c42-daca-42bc-ba9e-68191d659da1' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='a5853cd0-4b68-4166-965b-1586c0357f90'
------------------------------------------------------------------------------------------------




**Prepare a payload for scoring using the deployed model.**

In [14]:
a = np.array([xvalues[0]])
a.shape

(1, 31)

**8. Check the deployment for scoring using the prepared payload.**

In [15]:
model_endpoint_url = client.deployments.get_scoring_url( model_deployment_details )

payload = { "values" : a.tolist() }
client.deployments.score( model_endpoint_url, payload )

{'fields': ['prediction', 'prediction_classes', 'probability'],
 'values': [[[0.00010096141340909526], [0], [0.00010096141340909526]]]}